# Earth Engine Stuff

In [3]:
import ee
import geemap
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from pyproj import CRS
from cartopy import crs as ccrs
from geodatasets import get_path
import rioxarray as rxr 
import matplotlib.patches as mpatches
import time
import datetime

from osgeo import gdal
gdal.SetConfigOption('SHAPE_RESTORE_SHX', 'YES')

ee.Authenticate()
ee.Initialize(project='karlzam')


path = "/Users/kzammit/Documents/VIIRS-data-GEE/"

## Masks?

In [2]:
# A Sentinel-2 surface reflectance image.
img = ee.Image('COPERNICUS/S2_SR/20210109T185751_20210109T185931_T10SEG')
true_color_viz = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 2700,
    'gamma': 1.3,
}
display('Sentinel-2 image', img)
m = geemap.Map()
m.set_center(-122.36, 37.47, 10)
m.add_layer(img, true_color_viz, 'Sentinel-2 image')

# Get masks for all image bands each band has an independent mask.
# Valid pixels are value 1, invalid are 0.
multi_band_mask_img = img.mask()
display('Multi-band mask image', multi_band_mask_img)
m.add_layer(multi_band_mask_img, None, 'Multi-band mask image')

# Get the mask for a single image band.
single_band_mask_img = img.select('B1').mask()
display('Single-band mask image', single_band_mask_img)
m.add_layer(single_band_mask_img, None, 'Single-band mask image')
m

'Sentinel-2 image'

'Multi-band mask image'

'Single-band mask image'

Map(center=[37.47, -122.36], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchData…

## Loading Piyushs Script

In [4]:
################################################################################
# define areas of area of interest

# general bounding box
area_of_interest = ee.Geometry.Rectangle([-173, 24, -52, 72], 'EPSG:4326', False)

# North America
countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017").filter(ee.Filter.inList("wld_rgn", ['North America'])).filter(ee.Filter.inList("country_na", ['Canada', 'United States', 'United States (Alaska)']))

# create image mask for north america (this is faster for subsetting than Using
# original featurecollection "countries")
countries_mask = countries.map(
    lambda feature: feature.set('n', 1)
).reduceToImage(
    properties = ['n'],
    reducer = ee.Reducer.first()
).mask()

In [5]:
m.add_layer(countries_mask, None, 'Countries mask')
m

Map(bottom=13086.0, center=[36.721273880045004, -117.02087402343751], controls=(WidgetControl(options=['positi…

In [6]:
canada = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017").filter(ee.Filter.inList("wld_rgn", ['North America'])).filter(ee.Filter.inList("country_na", ['Canada']))
unitedstates = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017").filter(ee.Filter.inList("wld_rgn", ['North America'])).filter(ee.Filter.inList("country_na", ['United States', 'United States (Alaska)']))

canada_mask = canada.map(
    lambda feature: feature.set('n', 1)
).reduceToImage(
    properties = ['n'],
    reducer = ee.Reducer.first()
).mask()

unitedstates_mask = unitedstates.map(
    lambda feature: feature.set('n', 1)
).reduceToImage(
    properties = ['n'],
    reducer = ee.Reducer.first()
).mask()